# Intro

Le but de ce notebook est de faire un premier essai de scoring des communes en prenant:
- Identifiant quelques critères clés (voir liste ci-dessous)
- Clusteriser ces critères par communes / groupement de communes / bassin
- Tentative de scoring (avec pondération) selon le contexte de l'accueilli

De manière simplifiée le but est de calculer un ScoreAccueilli qui soit une somme pondérée de score thématiques:
ScoreAccueilli = XEmploi x ScoreEmploi + XLogement x ScoreLogement + XEdu x ScoreEdu + Score Politique

Chaque score est normalement distribuée autour de la valeur 0

## Critères / Score considèrés pour l'aggrégation:



Il existe 2 types de critères: des critères absolus qui représentent l'attractivité intrinsèque d'un territoire, les critères relatifs qui représentent l'importance selon le profil de la personne accueillie.

### Critères absolus

Emploi
- Ratio d'emploi non pourvus / 1000 hbts dans la Zone d'Emploi
- Ratio d'emploi en tensions non pourvus / 1000 hbts dans la Zone d'emploi

Logement
- Logements vacants / logements dans la commune
- Résidences principales de type maison et/ou avec 5+ pièces

Mobilité
- ???

Services
- Concentration de services d'inclusion dans la commune

Education
- Concentration Ecoles / Crèches dans un rayon de X km
- Présence d'écoles avec risque de fermeture de classes dans un rayon de X Km

Politique
- Couleur politique du maire de la commune ? --> Pas sûr de comment scorer

### Relatifs

Géographique
- Distance commune proposée <-> lieu de résidence actuel de l'accueilli

Emploi
- Ratio d'emplois pour des familles d'emplois adressable


# Fetching all the relevant datasets

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely import wkt
from unidecode import unidecode
import json

## Geographie & Polygons

source: https://www.data.gouv.fr/fr/datasets/communes-et-villes-de-france-en-csv-excel-json-parquet-et-feather/
- code_insee: Code commune, Code INSEE, Code assigné par l’INSEE à la commune
- nom_standard: Nom normalisé de la commune, avec son article (ex: Le Havre)
- nom_sans_pronom: Nom de la commune, sans son article le cas échéant (ex: Havre)
- nom_a: Nom de la commune, précédé de la préposition à, au ou aux et de l’article de la commune, le cas échéant (ex: au Havre)
- nom_de: Nom de la commune, précédé de la préposition d’, de, du ou des et de l’article de la commune, le cas échéant (ex: du Havre)
- nom_sans_accent: Nom de la commune sans accent, caractères spéciaux ou espaces
- nom_standard_majuscule: Nom de la commune en majuscule (ex: LE HAVRE)
- typecom: Type de la commune en version abrégée (COM, COMA, COMD, ARM)
- typecom_texte: Type de la commune en version textuelle
- reg_code: Code région assigné par l’INSEE à la région de la commune
- reg_nom: Nom de la région où est située la commune
- dep_code: Code département assigné par l’INSEE au département de la commune
- dep_nom: Nom du département où est située la commune
- canton_code: Code Canton de la commune
- canton_nom: Nom du canton de la commune
- epci_code: Code EPCI (établissements publics de coopération intercommunale) assigné par l’INSEE à la région de la commune
- epci_nom: Nom de l’EPCI où est située la commune
- code_postal: Code postal principal la commune
- codes_postaux: Codes postaux rattachés à la commune
- academie_code: Code de l’académie de rattachement des écoles de la commune
- academie_nom: Nom de l’académie de rattachement
- zone_emploi: Zone d’emploi de la commune, défini par l’INSEE
- code_insee_centre_zone_emploi: Code INSEE de la commune centre de la zone d’emploi
- code_unite_urbaine: Code INSEE de l’unité urbaine (agglomération). Si la commune est hors unité urbaine, il s’agit du code du département suivi par trois 0.
- nom_unite_urbaine: "Nom de l’unité urbaine (nom de l’agglomération)
- taille_unite_urbain: "Taille de l’unité urbaine
- type_commune_unite_urbain: "Type de commune (Hors unité urbaine ou Unité urbaine)
- statut_commune_unite_urbain: "Place de la commune dans l’unité urbaine (H: Hors unité urbaine, C: Ville-centre, B: Commune banlieue, I: Ville isolée)
- population: Population municipale
- superficie_hectare: Superficie de la commune, en hectare
- superficie_km2: Superficie de la commune, en km2
- densite: Densité de la commune, en habitant au km2
- altitude_moyenne: Altitude moyenne, en m
- altitude_minimale: Altitude minimale, en m
- altitude_maximale: Altitude maximale, en m
- latitude_mairie: Latitude de la mairie
- longitude_mairie: Longitude de la mairie
- latitude_centre: Latitude du centroïde du territoire communale
- longitude_centre: Longitude du centroïde du territoire communale
- grille_densite: Grille communale de densité à 7 niveaux, selon l’INSEE
- grille_densite_texte: Texte de la grille communale de densité à 7 niveaux, selon l’INSEE
- niveau_equipements_services: Niveau des équipements et des services, selon l’INSEE (de 0 à 4)
- niveau_equipements_services_texte: Texte du niveau des équipements et des services
- gentile: Gentilé (nom des habitants)
- url_wikipedia: URL de la page wikipédia de la commune
- url_villedereve: URL de la page Ville de rêve de la commune

In [3]:
#La base de villes de rêve aggrège beaucoup de données utiles pour les recoupement
data = json.load(open('../csv_large/communes-france-avec-polygon-2025.json'))
df = pd.DataFrame(data['data'])
df = df[['code_insee','nom_standard','typecom','reg_code','reg_nom','dep_code','dep_nom','epci_code','epci_nom','niveau_equipements_services',
         'academie_code','code_postal','codes_postaux','type_commune_unite_urbaine','population','code_insee_centre_zone_emploi','zone_emploi',
         'latitude_mairie','longitude_mairie']].copy()
df.rename(columns={'code_insee':'codgeo','nom_standard':'libgeo','code_insee_centre_zone_emploi':'codze'}, inplace=True)
df.set_index('codgeo', inplace=True)
# les codes postaux de Paris sont manquant
df.at['75056', 'codes_postaux'] = '75001, 75002, 75003, 75004, 75005, 75006, 75007, 75008, 75009, 75010, 75011, 75012, 75013, 75014, 75015, 75016, 75017, 75018, 75019, 75020, 75116'


In [4]:
polygons = pd.read_csv('../csv/communes-avec-polygons-json-2021.csv').drop(columns={'polygon_as_json'})
geo = pd.merge(df, polygons, on='codgeo', how='left')

In [5]:
def safe_load(x):
    try:
        return wkt.loads(x)
    except:
        return None

geo.polygon = geo.polygon.apply(safe_load)

In [6]:
geo = gpd.GeoDataFrame(geo)
geo.set_geometry(geo.polygon, crs='EPSG:4326', inplace=True)
del df
del polygons

In [6]:
voisins = pd.read_csv('../csv/communes_adjacentes_2022.csv')
voisins['codgeo_voisins'] = voisins.insee_voisins.str.split('|')
voisins = voisins[['insee','codgeo_voisins','nb_voisins']]
voisins.rename({'insee':'codgeo'}, inplace=True, axis=1)
voisins.head()

,codgeo,codgeo_voisins,nb_voisins
0,01001,"[01412, 01093, 01028, 01146, 01351, 01188]",6
1,01002,"[01056, 01277, 01384, 01007, 01363, 01199]",6
2,01004,"[01384, 01421, 01041, 01345, 01089, 01007, 01149]",7
3,01005,"[01382, 01207, 01261, 01362, 01318, 01398, 01446]",7
4,01006,"[01358, 01110, 01117, 01216, 01233, 01190]",6


## Population

In [7]:
pdf = pd.read_csv('../csv/insee-estimation-population-2024.csv')
pdf['population'] = 0
pdf['population'] = pdf['population'].case_when([
    (pdf.eval("p21_pop > 0"), pdf.p21_pop),
    (pdf.eval("p20_pop > 0"), pdf.p20_pop),
    (pdf.eval("p19_pop > 0"), pdf.p19_pop),
    (pdf.eval("p18_pop > 0"), pdf.p18_pop),
    (pdf.eval("p17_pop > 0"), pdf.p17_pop),
    (pdf.eval("p16_pop > 0"), pdf.p16_pop)
])
pop = pdf[['codgeo','libgeo','reg','dep','cv','population']]
pop.head()

,codgeo,libgeo,reg,dep,cv,population
0,85062,Châteauneuf,52,85,8502,1134.0
1,58300,Urzy,26,58,5808,1742.0
2,70137,Chassey-lès-Montbozon,43,70,7012,225.0
3,51649,Vitry-le-François,21,51,5123,11454.0
4,78638,Vaux-sur-Seine,11,78,7811,5083.0


## Emploi

In [8]:
#  Communes <-> Zones d'emploi 2020
## https://statistiques-locales.insee.fr/#c=zonage
## Choisir "Comparer deux Zonages, ZOnage 1 = Commune, Zonage 2 = Zones d'emploi 2020. Vue Tableau > Action > Exporter"
ze = pd.read_csv('../csv/Export_Communes_Zones_Emploi_2020.csv')
ze.codze2020 = ze.codze2020.astype('Int64').astype('string')
ze.head()

,codgeo,libgeo,ze_2020,codze2020,libze2020
0,01001,L'Abergement-Clémenciat,8405 - Bourg en Bresse,8405,Bourg en Bresse
1,01002,L'Abergement-de-Varey,8405 - Bourg en Bresse,8405,Bourg en Bresse
2,01004,Ambérieu-en-Bugey,8405 - Bourg en Bresse,8405,Bourg en Bresse
3,01005,Ambérieux-en-Dombes,8434 - Villefranche-sur-Saône,8434,Villefranche-sur-Saône
4,01006,Ambléon,8404 - Belley,8404,Belley


In [9]:
#  Communes <-> Bassins d'emploi 2021 de France Travail (=/= des Zones d'emploi de l'INSEE)
# https://statistiques.francetravail.org/bmo/static/methode_2021


be = pd.read_csv('../csv/France_Travail_Bassin_Emploi_2021.csv')
be.drop(['reg', 'dep'], axis=1, inplace=True)
be.codbe = be.codbe.astype('Int64').astype('string')
be.codgeo = np.where(be.iloc[:,0].str.len() == 4, '0' + be.iloc[:,0], be.iloc[:,0])
be.head()

,codgeo,codbe,libbe
0,63113,8457,CLERMONT FERRAND
1,63213,8457,CLERMONT FERRAND
2,63200,8457,CLERMONT FERRAND
3,63099,8457,CLERMONT FERRAND
4,63204,8457,CLERMONT FERRAND


In [10]:
#population par Bassin d'emploi
pop_be = pd.merge(pop, be, on='codgeo', how='left').groupby('codbe').sum('population')
pop_be.drop('reg', axis=1, inplace=True)
pop_be.rename({'population':'pop_be'}, axis=1, inplace=True)
pop_be.head()

,pop_be
codbe,
1107,2133111.0
1108,201762.0
1109,144581.0
1115,176665.0
1116,150292.0


In [11]:
# Besoin en emploi pas bassin de vie / zone d'emploi
# met	Nombre de projet de recrutement 
# xmet	Nombre de projets de recrutement jugés difficiles
# smet	Nombre de projets de recrutement saisonniers
bmo_be_france = pd.read_csv('../csv/france_travail_base_open_data_BMO_2025.csv')
bmo_be_france = bmo_be_france.rename(columns={'Code métier BMO':'codfap','Nom métier BMO':'libfap','REG': 'code_region', 'Dept':'code_dep','BE25':'codbe', 'NOMBE25':'libbe'})
bmo_be_france = bmo_be_france[['codbe','codfap','libfap','code_region','code_dep','met','xmet','smet']]
bmo_be_france.codbe = bmo_be_france.codbe.astype('string') 
bmo_be_france.met = bmo_be_france.met.replace('*','0').astype('Int64')
bmo_be_france.xmet = bmo_be_france.xmet.replace('*','0').astype('Int64')
bmo_be_france.smet = bmo_be_france.smet.replace('*','0').astype('Int64')
bmo_be_france.head()

,codbe,codfap,libfap,code_region,code_dep,met,xmet,smet
0,101,A0X40,Agriculteurs,1,971,227,80,128
1,102,A0X40,Agriculteurs,1,971,91,42,82
2,105,A0X40,Agriculteurs,1,971,57,52,7
3,113,A0X40,Agriculteurs,1,971,26,0,26
4,116,A0X40,Agriculteurs,1,971,0,0,0


In [12]:
emploi_tension = pd.read_csv('../csv/metiers_en_tension_mars_2024.csv')
emploi_tension = emploi_tension.rename({'Region':'nom_region','Code FAP':'codfap','Famille Professionnelle':'libfap'},axis=1)
emploi_tension.replace({'Ile-de-France':'Île-de-France'}, inplace=True)

regions_insee = pd.read_csv('../csv/insee_region_2022.csv')
regions_insee.rename({'LIBELLE':'nom_region','REG':'code_region'}, inplace=True, axis=1)
regions_insee = regions_insee[['nom_region','code_region']]

emploi_tension = pd.merge(emploi_tension, regions_insee, on='nom_region')
emploi_tension.head()

,nom_region,libfap,codfap,code_region
0,Auvergne-Rhône-Alpes,Agriculteurs salariés,A0Z40,84
1,Auvergne-Rhône-Alpes,Éleveurs salariés,A0Z41,84
2,Auvergne-Rhône-Alpes,Maraîchers; horticulteurs salariés,A1Z40,84
3,Auvergne-Rhône-Alpes,Viticulteurs; arboriculteurs salariés,A1Z42,84
4,Auvergne-Rhône-Alpes,Agents de maîtrise et assimilés des industries...,E2Z80,84


In [13]:
#identification des BMO pour métiers en tension vs autres
bmo_be_france  = pd.merge(bmo_be_france, emploi_tension.drop('libfap', axis=1), on=['code_region', 'codfap'], how='left')
bmo_be_france.nom_region  = np.where(bmo_be_france.nom_region.isna(), 0, bmo_be_france.met)
bmo_be_france.rename({'nom_region':'met_tension'}, axis=1, inplace=True)
bmo_be_france.head()

,codbe,codfap,libfap,code_region,code_dep,met,xmet,smet,met_tension
0,101,A0X40,Agriculteurs,1,971,227,80,128,0
1,102,A0X40,Agriculteurs,1,971,91,42,82,0
2,105,A0X40,Agriculteurs,1,971,57,52,7,0
3,113,A0X40,Agriculteurs,1,971,26,0,26,0
4,116,A0X40,Agriculteurs,1,971,0,0,0,0


In [14]:
#Aggrégation par bassin d'emploi (codbe)
bmo_be_france_agg = bmo_be_france.groupby(['codbe'], as_index=False).aggregate({'met':'sum','met_tension':'sum', 'codfap':'count'})
bmo_be_france_agg.head()

,codbe,met,met_tension,codfap
0,101,2855,0,129
1,102,3129,0,109
2,105,5874,0,148
3,1107,131328,346,202
4,1108,3762,42,132


In [15]:
#Ajout des top N metiers demandés
n=10
bmo_be_france_topn = bmo_be_france.sort_values('met', ascending=False).groupby('codbe').head(n)
bmo_be_france_topn = bmo_be_france_topn[['codbe','codfap','libfap']].groupby('codbe').agg(list)
bmo_be_france_topn.rename(columns={'codfap':'be_codfap_top','libfap':'be_libfap_top'}, inplace=True)

bmo_be_france_agg = pd.merge(bmo_be_france_agg, bmo_be_france_topn, on='codbe', how='left')
bmo_be_france_agg.head()

,codbe,met,met_tension,codfap,be_codfap_top,be_libfap_top
0,101,2855,0,129,"[A0X40, V0X60, R1X60, S2X61, V1X80, S1X20, T2A...","[Agriculteurs, Aides-soignants, Vendeurs en pr..."
1,102,3129,0,109,"[T2A60, U1X91, V5X81, S1X20, S2X61, A1X41, J3X...","[Aides à domicile et auxiliaires de vie, Artis..."
2,105,5874,0,148,"[R0X60, S1X20, T3X61, U1X91, V5X81, L0X60, L2X...","[Employés de libre service, Aides de cuisine e..."
3,1107,131328,346,202,"[S2X61, S1X20, L2X60, U1X91, U1X80, M2X90, S2X...","[Serveurs de cafés restaurants, Aides de cuisi..."
4,1108,3762,42,132,"[S1X20, S2X61, T2A60, R0X60, V0X60, V5X81, A1X...",[Aides de cuisine et employés polyvalents de l...


In [16]:
#Ajout de la population du bassin d'emploi
bmo_be_france_agg = bmo_be_france_agg.merge(pop_be, on='codbe', how='left')
bmo_be_france_agg.dropna(inplace=True) # Restricte à la métropole
bmo_be_france_agg.head()

,codbe,met,met_tension,codfap,be_codfap_top,be_libfap_top,pop_be
3,1107,131328,346,202,"[S2X61, S1X20, L2X60, U1X91, U1X80, M2X90, S2X...","[Serveurs de cafés restaurants, Aides de cuisi...",2133111.0
4,1108,3762,42,132,"[S1X20, S2X61, T2A60, R0X60, V0X60, V5X81, A1X...",[Aides de cuisine et employés polyvalents de l...,201762.0
5,1109,3483,94,122,"[T4X60, U1X91, T2A60, R0X60, A1X41, G0A40, S1X...","[Agents d'entretien de locaux, Artistes (musiq...",144581.0
6,1115,4365,40,152,"[J0X33, V5X81, S1X20, V1X80, V0X60, S2X61, R0X...",[Magasiniers et préparateurs de commandes peu ...,176665.0
7,1116,2955,29,116,"[A0X40, V5X81, U1X91, A1X42, V0X60, M2X90, T1X...","[Agriculteurs, Professionnels de l'animation s...",150292.0


## Formation

In [17]:
# source: https://www.data.gouv.fr/fr/datasets/liste-publique-des-organismes-de-formation-l-6351-7-1-du-code-du-travail/
# We need to merge this data with the main dataframe using the code postal
centres_formations = pd.read_csv('../csv/public_ofs_v2.csv', delimiter=';', low_memory=False)
centres_formations['codes_formations'] = centres_formations[
    ['informationsDeclarees.specialitesDeFormation.codeSpecialite1',
     'informationsDeclarees.specialitesDeFormation.codeSpecialite2',
     'informationsDeclarees.specialitesDeFormation.codeSpecialite3']
     ].apply(lambda row: [int(x) for x in row if pd.notna(x)], axis=1)
centres_formations['noms_formations']= centres_formations[
    ['informationsDeclarees.specialitesDeFormation.libelleSpecialite1',
     'informationsDeclarees.specialitesDeFormation.libelleSpecialite2',
     'informationsDeclarees.specialitesDeFormation.libelleSpecialite3']
     ].apply(lambda row: [x for x in row if pd.notna(x)], axis=1)
centres_formations = centres_formations.rename({'adressePhysiqueOrganismeFormation.codePostal':'codepostal'}, axis=1)
centres_formations = centres_formations.dropna(subset=['codepostal'])
centres_formations = centres_formations[['siren','codepostal','codes_formations', 'noms_formations']]

centres_formations.codepostal = centres_formations.codepostal.astype(int).astype(str)

In [18]:
# for each codgeo gathering all the possible codes formations
formations = geo[['codgeo', 'codes_postaux']].copy()
formations.codes_postaux = formations.codes_postaux.str.split(',')
formations = pd.merge(formations.explode('codes_postaux'), centres_formations, left_on='codes_postaux', right_on='codepostal', how='inner').drop(columns={'codepostal','codes_postaux'})
formations = formations.groupby('codgeo', as_index=False).aggregate({'codes_formations':'sum', 'noms_formations':'sum'})

#cleaning up the list of codes formations
def codes_formations_cleanup(df):
    myset = set(df.tolist())
    myset.discard(0)
    return list(myset)

formations.codes_formations = formations.codes_formations#.apply(codes_formations_cleanup)

## Education

In [19]:
effectifs_ecoles = pd.read_parquet('../csv/fr-en-ecoles-effectifs-nb_classes.parquet')
effectifs_ecoles.rename(columns={
    'nombre_eleves_preelementaire_hors_ulis':'mat_ct',
    'nombre_eleves_cp_hors_ulis':'cp_ct',
    'nombre_eleves_ce1_hors_ulis':'ce1_ct',
    'nombre_eleves_ce2_hors_ulis':'ce2_ct',
    'nombre_eleves_cm1_hors_ulis':'cm1_ct',
    'nombre_eleves_cm2_hors_ulis':'cm2_ct',
}, inplace=True)
# On effectue notre merge sur le nom de la commune (horrible...)
effectifs_ecoles = pd.merge(effectifs_ecoles[effectifs_ecoles.rentree_scolaire == '2023'], pop, left_on='commune', right_on=pop.libgeo.apply(unidecode).str.upper(), how='left')
effectifs_ecoles.reset_index()
effectifs_ecoles = effectifs_ecoles[['codgeo','numero_ecole','mat_ct','cp_ct','ce1_ct','ce2_ct','cm1_ct','cm2_ct']]
effectifs_ecoles.head()

,codgeo,numero_ecole,mat_ct,cp_ct,ce1_ct,ce2_ct,cm1_ct,cm2_ct
0,04001,0040057R,33,8,17,10,16,19
1,06001,0040057R,33,8,17,10,16,19
2,04004,0040058S,24,10,4,6,0,0
3,04006,0040059T,5,3,2,1,4,3
4,04008,0040064Y,42,8,15,15,19,19


In [20]:
seuil_risque = 0
seuils_fermeture = [15,23,45,70,93,116,139,162,185,208,231,254]
def risque_fermeture(effectif):
    seuil=seuils_fermeture[0]
    for i in seuils_fermeture:
        if i < effectif:
            seuil = i 
    if effectif - seuil < seuil_risque:
        return 1
    else:
        return 0

effectifs_ecoles['risque_fermeture'] = (effectifs_ecoles.mat_ct.apply(risque_fermeture) 
                                        + effectifs_ecoles.cp_ct.apply(risque_fermeture)
                                        + effectifs_ecoles.ce1_ct.apply(risque_fermeture)
                                        + effectifs_ecoles.ce2_ct.apply(risque_fermeture)
                                        + effectifs_ecoles.cm1_ct.apply(risque_fermeture)
                                        + effectifs_ecoles.cm2_ct.apply(risque_fermeture))
#or effectifs_ecoles.cp.apply(risque_fermeture) or effectifs_ecoles.ce1.apply(risque_fermeture) or effectifs_ecoles.ce2.apply(risque_fermeture) or effectifs_ecoles.cm1.apply(risque_fermeture) or effectifs_ecoles.cm2.apply(risque_fermeture) 
 
effectifs_ecoles.head(20)

,codgeo,numero_ecole,mat_ct,cp_ct,ce1_ct,ce2_ct,cm1_ct,cm2_ct,risque_fermeture
0,04001,0040057R,33,8,17,10,16,19,2
1,06001,0040057R,33,8,17,10,16,19,2
2,04004,0040058S,24,10,4,6,0,0,5
3,04006,0040059T,5,3,2,1,4,3,6
4,04008,0040064Y,42,8,15,15,19,19,1
5,04013,0040065Z,0,9,17,11,16,8,4
6,04018,0040067B,39,12,11,12,9,13,5
7,04019,0040070E,45,0,0,0,0,0,5
8,04019,0040159B,30,11,10,18,8,12,4
9,04019,0040068C,0,19,18,21,18,28,1


In [21]:
effectifs_ecoles_agg = effectifs_ecoles[['codgeo','risque_fermeture','numero_ecole']].groupby(['codgeo']).agg({'risque_fermeture':'sum','numero_ecole':'count'})
effectifs_ecoles_agg.rename(columns={'numero_ecole':'ecoles_ct'}, inplace=True)
effectifs_ecoles_agg.head(20)

,risque_fermeture,ecoles_ct
codgeo,,
01004,17,7
01005,0,1
01007,6,2
01008,5,1
01010,4,1
01011,27,6
01012,6,1
01013,6,1
01014,15,4


Now let's prepare a mini dataset for the annuaires ecoles that can be used later in the streamlit app

In [22]:
# Source:https://www.data.gouv.fr/fr/datasets/annuaire-de-leducation/
ecoles = gpd.read_file('../csv_large/fr-en-annuaire-education.geojson')
col = ['identifiant_de_l_etablissement', 'nom_etablissement', 'type_etablissement', 'statut_public_prive', 'code_commune', 'nom_commune', 'code_region','code_academie', 'ecole_maternelle', 'ecole_elementaire', 'voie_generale', 'nombre_d_eleves','geometry']
#ecoles.geometry = ecoles.geometry.apply(wkt.loads)
ecoles.dropna(subset=['geometry'], inplace=True)
ecoles = ecoles[col].set_geometry('geometry', crs='EPSG:4326')
ecoles.to_parquet('../csv/annuaire_ecoles_france_mini.parquet')

In [23]:
ecoles.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 68792 entries, 0 to 69569
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   identifiant_de_l_etablissement  68792 non-null  object  
 1   nom_etablissement               68792 non-null  object  
 2   type_etablissement              68536 non-null  object  
 3   statut_public_prive             66748 non-null  object  
 4   code_commune                    68792 non-null  object  
 5   nom_commune                     68792 non-null  object  
 6   code_region                     68792 non-null  object  
 7   code_academie                   68792 non-null  object  
 8   ecole_maternelle                48701 non-null  float64 
 9   ecole_elementaire               48701 non-null  float64 
 10  voie_generale                   11214 non-null  object  
 11  nombre_d_eleves                 58108 non-null  float64 
 12  geometry       

## Logement

Source: https://www.insee.fr/fr/statistiques/8268838

In [24]:
logements = gpd.read_file('../csv_large/base-ic-logement-2021.csv')
logements = logements[['COM','P21_MEN','P21_PMEN','P21_LOG','P21_RP','P21_LOGVAC','P21_RSECOCC','P21_RP_5PP', 'P21_NBPI_RP','P21_NBPI_RPMAISON']].rename({'COM':'codgeo','P21_MEN':'men_ct','P21_PMEN':'men_pop','P21_LOG':'log_total','P21_RP':'log_rp','P21_LOGVAC':'log_vac','P21_MAISON':'log_maison','P21_RSECOCC':'log_rsec','P21_RP_5PP':'rp_5+pieces','P21_NBPI_RP':'pieces_rp', 'P21_NBPI_RPMAISON':'pieces_maison'},axis=1)
for column in logements.columns:
    if column != 'codgeo':
        logements[column] = logements[column].astype('float')

In [25]:
logements_agg = logements.groupby('codgeo').agg({
    'men_ct':'sum',
    'men_pop':'sum',
    'log_total':'sum',
    'log_rp':'sum',
    'log_vac':'sum',
    'log_rsec':'sum',
    'rp_5+pieces':'sum',
    'pieces_maison':'sum',
    'pieces_rp':'sum'
    }).reset_index()
del logements
logements_agg.head()

,codgeo,men_ct,men_pop,log_total,log_rp,log_vac,log_rsec,rp_5+pieces,pieces_maison,pieces_rp
0,01001,341.234562,832.000000,372.387494,341.234562,17.445642,13.707290,212.547081,1700.459399,1723.390832
1,01002,115.722510,267.000000,174.938672,115.722510,14.255743,44.960419,72.242622,584.301613,590.738459
2,01004,6932.839198,14352.998425,7846.021789,6932.839198,775.203248,137.979344,2077.545519,14586.137226,26714.019761
3,01005,794.753025,1897.000000,898.442947,794.753025,96.129199,7.560724,409.132882,3042.374475,3683.448628
4,01006,56.500000,113.000000,73.027778,56.500000,7.083333,9.444444,32.710526,270.605263,275.561404


In [39]:
#Source: https://www.statistiques.developpement-durable.gouv.fr/54-millions-de-logements-locatifs-sociaux-en-france-au-1er-janvier-2024#:~:text=Au%201er%20janvier%202024%2C%20la,11%20400%20ont%20%C3%A9t%C3%A9%20vendus.
logements_sociaux = pd.read_csv('../csv/export_parc_logements_sociaux_jan_2024.csv', usecols=['DEPCOM_ARM', 'nb_vacants', 'nb_vides', 'nb_ls'])
# logements_sociaux = logements_sociaux[]
logements_sociaux.rename(columns={
    'DEPCOM_ARM':'codgeo', 
    'nb_vacants':'log_soc_vacants',
    'nb_vides':'log_soc_vides',
    'nb_ls':'log_soc_total'
    }, inplace=True)
logements_sociaux.fillna(0, inplace=True)
logements_sociaux['log_soc_inoccupes'] = logements_sociaux['log_soc_vacants'].astype(int) + logements_sociaux['log_soc_vides'].astype(int)


## Soutien

Source: https://www.data.gouv.fr/fr/datasets/referentiel-de-loffre-dinsertion-sociale-et-professionnelle-data-inclusion/


In [49]:
incl = json.load(open('../csv_large/services-inclusion-2025-04-28.geojson'))
incl_norm = pd.json_normalize(data=incl, record_path='features')
incl_norm.rename(columns={
    'properties.nom':'nom',
    'properties.structure_id':'structure_id',
    'properties.source':'source',
    'properties.types':'types',
    'properties.thematiques':'thematiques',
    'properties.frais':'frais',
    'properties.profils':'profils',
    'properties.longitude':'longitude',
    'properties.latitude':'latitude',
    'properties.adresse':'adresse',
    'properties.code_insee':'codgeo',
    'properties.presentation_resume':'presentation_resume',
    }, inplace=True)

incl_norm = incl_norm[['nom','structure_id','source','types','thematiques','frais','profils','longitude','latitude','adresse','codgeo','presentation_resume']]

In [51]:
incl_exploded=incl_norm.explode('thematiques')
incl_exploded.dropna(subset='thematiques', inplace=True, ignore_index=True)
thematiques_split_df = pd.DataFrame(incl_exploded["thematiques"].str.split('--', expand=True).values, columns=['categorie', 'service'])
thematiques_split_df['service'].fillna('-', inplace=True)
incl_exploded = pd.concat([incl_exploded, thematiques_split_df], axis=1)
incl_exploded['geometry'] = gpd.points_from_xy(incl_exploded.longitude, incl_exploded.latitude)
incl_exploded = gpd.GeoDataFrame(incl_exploded, geometry='geometry', crs='EPSG:4326')
incl_exploded.to_parquet('../csv/odis_services_incl_exploded.parquet')

/tmp/ipykernel_404867/3672608668.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  thematiques_split_df['service'].fillna('-', inplace=True)


In [29]:
# services_incl_source_agg = incl_exploded.value_counts(['source', 'codgeo']).rename('svc_incl_count').reset_index()
services_incl_agg = incl_exploded.value_counts(['codgeo']).rename('svc_incl_count').reset_index()
# services_incl_agg.rename(columns={'code_insee':'codgeo'}, inplace=True)
del incl_exploded, incl_norm
services_incl_agg.head()

,codgeo,svc_incl_count
0,75056,4428
1,97416,3510
2,13055,2541
3,29019,1881
4,31555,1656


In [43]:
pol = pd.read_csv('../csv/rne-enrichi-couleur-politique.csv')
pol.rename(columns={'cog_commune':'codgeo'}, inplace=True)
pol = pol[['codgeo','nuance_politique','famille_nuance']]
pol.head()

,codgeo,nuance_politique,famille_nuance
0,01001,NC,Non classé
1,01002,NC,Non classé
2,01004,LDVC,Centre
3,01005,NaN,NaN
4,01006,NC,Non classé


In [45]:
# Découpage grossier des communes par nuance politique
pol['pol_num'] = pol.famille_nuance.case_when([
    (pol.famille_nuance.isin(['Extrême droite']),0),
    (pol.famille_nuance.isin(['Droite']),0.25),
    (pol.famille_nuance.isin(['Centre','Courants politiques divers','Non classé']),0.5),
    (pol.famille_nuance.isna(),0.5),
    (pol.famille_nuance.isin(['Gauche']),1),
])
# pol.groupby(['pol_num']).count()

# Mega Merge

In [ ]:
# Merging all the tables
odis = pd.merge(geo, be, on='codgeo', how='left')
odis = pd.merge(odis, bmo_be_france_agg, on='codbe', how='left')
odis = pd.merge(odis, formations, on='codgeo', how='left')
odis = pd.merge(odis, services_incl_agg, on='codgeo', how='left')
odis = pd.merge(odis, logements_agg[['codgeo','log_total','log_rp','log_vac','rp_5+pieces']], on='codgeo', how='left')
odis = pd.merge(odis, logements_sociaux, on='codgeo', how='left')
odis = pd.merge(odis, effectifs_ecoles_agg, on='codgeo', how='left')
odis = pd.merge(odis, pol, on='codgeo', how='left')
odis = pd.merge(odis, voisins, on='codgeo', how='left')



NameError: name 'pd' is not defined

# Export

In [ ]:
#odis.set_crs("EPSG:4326")
odis.to_parquet('../csv/odis_june_2025_jacques.parquet')

In [35]:
odis.head()

,codgeo,libgeo,typecom,reg_code,reg_nom,dep_code,dep_nom,epci_code,epci_nom,niveau_equipements_services,...,nb_vides,nb_total,nb_inoccupes,risque_fermeture,ecoles_ct,nuance_politique,famille_nuance,pol_num,codgeo_voisins,nb_voisins
0,01001,L'Abergement-Clémenciat,COM,84,Auvergne-Rhône-Alpes,01,Ain,200069193,CC de la Dombes,0.0,...,0.0,32.0,1.0,NaN,NaN,NC,Non classé,0,"[01412, 01093, 01028, 01146, 01351, 01188]",6.0
1,01002,L'Abergement-de-Varey,COM,84,Auvergne-Rhône-Alpes,01,Ain,240100883,CC de la Plaine de l'Ain,0.0,...,NaN,NaN,NaN,NaN,NaN,NC,Non classé,0,"[01056, 01277, 01384, 01007, 01363, 01199]",6.0
2,01004,Ambérieu-en-Bugey,COM,84,Auvergne-Rhône-Alpes,01,Ain,240100883,CC de la Plaine de l'Ain,3.0,...,59.0,2109.0,155.0,17.0,7.0,LDVC,Centre,0,"[01384, 01421, 01041, 01345, 01089, 01007, 01149]",7.0
3,01005,Ambérieux-en-Dombes,COM,84,Auvergne-Rhône-Alpes,01,Ain,200042497,CC Dombes Saône Vallée,1.0,...,2.0,113.0,5.0,0.0,1.0,NaN,NaN,0,"[01382, 01207, 01261, 01362, 01318, 01398, 01446]",7.0
4,01006,Ambléon,COM,84,Auvergne-Rhône-Alpes,01,Ain,200040350,CC Bugey Sud,0.0,...,NaN,NaN,NaN,NaN,NaN,NC,Non classé,0,"[01358, 01110, 01117, 01216, 01233, 01190]",6.0


# Explorations